# 5001 Kaggle Competition

In [57]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [193]:
# load data in colab
data_root = '/content/drive/MyDrive/5001/kaggle competitions/data/'

train = pd.read_csv(os.path.join(data_root, 'train.csv'))
test = pd.read_csv(os.path.join(data_root, 'test.csv'))
sub = pd.read_csv(os.path.join(data_root, 'sample_submission.csv'))

In [160]:
# see the dimension and sample size
print(train.shape, test.shape)

(87, 13) (59, 12)


In [192]:
train.head()
# every attribute is numerical

,MO HLADR+ MFI (cells/ul),Neu CD64+MFI (cells/ul),CD3+T (cells/ul),CD8+T (cells/ul),CD4+T (cells/ul),NK (cells/ul),CD19+ (cells/ul),CD45+ (cells/ul),Age,Sex 0M1F,Mono CD64+MFI (cells/ul),label
0,3556.0,2489.0,265.19,77.53,176.55,0.00,4.20,307.91,52,0,7515.0,1
1,1906.0,134.0,1442.61,551.90,876.07,112.10,168.15,1735.48,20,1,1756.0,0
2,1586.0,71.0,1332.74,684.20,655.26,244.95,216.52,1820.04,28,1,1311.0,0
3,683.0,94.0,419.23,255.80,162.17,72.05,44.68,538.22,55,1,1443.0,0
4,1032.0,71.0,1102.72,480.27,625.30,188.78,130.77,1427.97,28,1,1542.0,0


In [ ]:
test.head()

,id,MO HLADR+ MFI (cells/ul),Neu CD64+MFI (cells/ul),CD3+T (cells/ul),CD8+T (cells/ul),CD4+T (cells/ul),NK (cells/ul),CD19+ (cells/ul),CD45+ (cells/ul),Age,Sex 0M1F,Mono CD64+MFI (cells/ul)
0,0,2843.0,156.0,1358.52,730.78,637.85,127.06,94.82,1588.62,45,1,3256.0
1,1,437.0,137.0,509.43,268.05,243.07,390.86,98.24,1002.76,51,1,491.0
2,2,826.0,82.0,1232.22,493.42,744.08,516.28,320.15,2200.58,32,0,1381.0
3,3,861.0,50.0,1512.86,925.51,590.07,380.25,25.80,1929.10,50,0,1377.0
4,4,1160.0,157.0,890.42,403.91,489.53,266.92,87.63,1251.52,43,0,1844.0


In [195]:
sub.head()

,id,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [194]:
# delete column 'id'
train = train.drop(columns='id')
test = test.drop(columns='id')

In [162]:
# check null values
train[train.isnull().T.any()]

,MO HLADR+ MFI (cells/ul),Neu CD64+MFI (cells/ul),CD3+T (cells/ul),CD8+T (cells/ul),CD4+T (cells/ul),NK (cells/ul),CD19+ (cells/ul),CD45+ (cells/ul),Age,Sex 0M1F,Mono CD64+MFI (cells/ul),label
39,NaN,NaN,1336.54,739.71,550.3,68.46,192.07,1615.68,21,0,NaN,0


In [163]:
test[test.isnull().T.any()]

,MO HLADR+ MFI (cells/ul),Neu CD64+MFI (cells/ul),CD3+T (cells/ul),CD8+T (cells/ul),CD4+T (cells/ul),NK (cells/ul),CD19+ (cells/ul),CD45+ (cells/ul),Age,Sex 0M1F,Mono CD64+MFI (cells/ul)


In [ ]:
# delete null values
train = train.dropna()

In [165]:
y = train['label']
x = train.drop(columns=['label'])

print("Shape of x:", x.shape)
print("Shape of y:", y.shape)

#X_norm = sc.fit_transform(X)

Shape of x: (86, 11)
Shape of y: (86,)


In [177]:
# get training and test set
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.2, random_state = 0)

# try random forest
clf = RandomForestClassifier()
clf.fit(x_train, y_train)
score = clf.score(x_valid, y_valid)
print(score)

0.9444444444444444


In [151]:
# grid research
parameters = {'criterion':('gini','entropy'), 'min_samples_split':[2,5,10],
             'min_samples_leaf':[1,2,5],'max_features':["auto", "sqrt", "log2"],
             'oob_score':[True,False],'class_weight':["balanced","balanced_subsample"]}

rf = RandomForestClassifier(200,random_state = 39)

clf = GridSearchCV(rf, parameters,cv = 10, scoring='accuracy')

clf.fit(x, y)

GridSearchCV(cv=10,
             estimator=RandomForestClassifier(n_estimators=200,
                                              random_state=39),
             param_grid={'class_weight': ['balanced', 'balanced_subsample'],
                         'criterion': ('gini', 'entropy'),
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 5],
                         'min_samples_split': [2, 5, 10],
                         'oob_score': [True, False]},
             scoring='accuracy')

In [186]:
y_pred_test = clf.predict(test)
sub['label'] = y_pred_test
sub.to_csv(os.path.join(data_root, 'sub1.csv'), index=False)